## Importing Libraries

In [286]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
## load the data

In [287]:
df = pd.read_json("data.json")
df['req'].to_json("req.json")

In [288]:
df

,req,isSafe
0,"{'baseUrl': '/crm/note', 'body': {'note': {'ti...",True
1,"{'baseUrl': '/crm/note', 'body': {'note': {'ti...",True
2,"{'baseUrl': '/crm/note', 'body': {'note': {'ti...",False
3,"{'baseUrl': '/crm/note', 'body': {'note': {'ti...",True
4,"{'baseUrl': '/crm/note', 'body': {'note': {'ti...",False
...,...,...
995,"{'baseUrl': '/crm/note', 'body': {'note': {'ti...",True
996,"{'baseUrl': '/crm/note', 'body': {'note': {'ti...",False
997,"{'baseUrl': '/crm/note', 'body': {'note': {'ti...",True
998,"{'baseUrl': '/crm/note', 'body': {'note': {'ti...",True


In [377]:
## Creating multiple json file and load the data to get the important informations only

In [111]:
df1 = pd.read_json("req.json")

In [112]:
df2 = np.transpose(df1)
df2.head()

,baseUrl,body,fresh,headers,hostname,ip,ips,originalUrl,params,path,protocol,query,secure,stale,subdomains,xhr
0,/crm/note,"{'note': {'title': 'Tina Johnson', 'desc': 'To...",False,"{'host': 'example.com', 'user-agent': 'insomni...",example.com,::ffff:117.99.96.244,[],/crm/note,{},/note,http,{},False,True,[crm],False
1,/crm/note,"{'note': {'title': 'Clayton Cooper', 'desc': '...",False,"{'host': 'example.com', 'user-agent': 'insomni...",example.com,::ffff:117.99.96.244,[],/crm/note,{},/note,http,{},False,True,[crm],False
2,/crm/note,"{'note': {'title': 'Curtis Wolfe', 'desc': 'Tu...",False,"{'host': 'example.com', 'user-agent': 'insomni...",example.com,::ffff:117.99.96.244,[],/crm/note,{},/note,http,{},False,True,[crm],False
3,/crm/note,"{'note': {'title': 'Laura Fisher', 'desc': 'St...",False,"{'host': 'example.com', 'user-agent': 'insomni...",example.com,::ffff:117.99.96.244,[],/crm/note,{},/note,http,{},False,True,[crm],False
4,/crm/note,"{'note': {'title': 'Tyler Santos', 'desc': 'Us...",False,"{'host': 'example.com', 'user-agent': 'insomni...",example.com,::ffff:117.99.96.244,[],/crm/note,{},/note,http,{},False,True,[crm],False


In [113]:
df2.shape

(1000, 16)

In [114]:
df2['body'].to_json("body.json")

In [115]:
body = pd.read_json("body.json")

In [116]:
note = np.transpose(body)

In [117]:
note["note"].to_json("note.json")
note = pd.read_json("note.json")
file = np.transpose(note)
file

,title,desc
0,Tina Johnson,Top recognize eat. Fact whom spend area thing ...
1,Clayton Cooper,As possible American many prepare four strong....
2,Curtis Wolfe,Tuesday Notes or 2 like 2 XSP Class
3,Laura Fisher,State third represent energy campaign not forg...
4,Tyler Santos,Us enjoy since. Time identify image position o...
...,...,...
995,Bridget Elliott,Recently crime before five thought bit. Card f...
996,James Rocha,"<img src=1 href=1 onerror=""javascript:alert(1)..."
997,Jasmine Gibson,Set nature they then low resource truth. Edge ...
998,Mr. Antonio Valdez DDS,Push case them such face suffer. Letter middle...


In [118]:
file['desc']

0      Top recognize eat. Fact whom spend area thing ...
1      As possible American many prepare four strong....
2                    Tuesday Notes or 2 like 2 XSP Class
3      State third represent energy campaign not forg...
4      Us enjoy since. Time identify image position o...
                             ...                        
995    Recently crime before five thought bit. Card f...
996    <img src=1 href=1 onerror="javascript:alert(1)...
997    Set nature they then low resource truth. Edge ...
998    Push case them such face suffer. Letter middle...
999    Family ready stay rule full than yet. Moment o...
Name: desc, Length: 1000, dtype: object

In [120]:
data = pd.concat([df2['fresh'],file],axis=1)

In [121]:
data.head()

,fresh,title,desc
0,False,Tina Johnson,Top recognize eat. Fact whom spend area thing ...
1,False,Clayton Cooper,As possible American many prepare four strong....
2,False,Curtis Wolfe,Tuesday Notes or 2 like 2 XSP Class
3,False,Laura Fisher,State third represent energy campaign not forg...
4,False,Tyler Santos,Us enjoy since. Time identify image position o...


In [122]:
df2["headers"].to_json("header.json")
header = pd.read_json("header.json")
header = np.transpose(header)
header

,host,user-agent,content-type,org_id,user_session_id,accept,content-length,user
0,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70,"{'name': 'sam', 'role': 'sales1'}"
1,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70,"{'name': 'sam', 'role': 'sales1'}"
2,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70,"{'name': 'sam', 'role': 'sales1'}"
3,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70,"{'name': 'sam', 'role': 'sales1'}"
4,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70,"{'name': 'sam', 'role': 'sales1'}"
...,...,...,...,...,...,...,...,...
995,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70,"{'name': 'sam', 'role': 'sales1'}"
996,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70,"{'name': 'sam', 'role': 'sales1'}"
997,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70,"{'name': 'sam', 'role': 'sales1'}"
998,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70,"{'name': 'sam', 'role': 'sales1'}"


In [123]:
header["user"].to_json("user.json")
user= pd.read_json("user.json")
user = np.transpose(user)
user

,name,role
0,sam,sales1
1,sam,sales1
2,sam,sales1
3,sam,sales1
4,sam,sales1
...,...,...
995,sam,sales1
996,sam,sales1
997,sam,sales1
998,sam,sales1


In [124]:
user['role'].value_counts()

sales1    1000
Name: role, dtype: int64

In [125]:
header.drop(['user'],axis=1,inplace=True)

In [126]:
header

,host,user-agent,content-type,org_id,user_session_id,accept,content-length
0,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
1,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
2,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
3,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
4,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
...,...,...,...,...,...,...,...
995,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
996,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
997,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
998,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70


In [127]:
data = pd.concat([data,header],axis=1)
data.head()

,fresh,title,desc,host,user-agent,content-type,org_id,user_session_id,accept,content-length
0,False,Tina Johnson,Top recognize eat. Fact whom spend area thing ...,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
1,False,Clayton Cooper,As possible American many prepare four strong....,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
2,False,Curtis Wolfe,Tuesday Notes or 2 like 2 XSP Class,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
3,False,Laura Fisher,State third represent energy campaign not forg...,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70
4,False,Tyler Santos,Us enjoy since. Time identify image position o...,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70


In [128]:
df2["xhr"].value_counts()

False    1000
Name: xhr, dtype: int64

In [129]:
df2.drop(['body',"headers","fresh"],axis=1,inplace=True)

In [130]:
df2.head(2)

,baseUrl,hostname,ip,ips,originalUrl,params,path,protocol,query,secure,stale,subdomains,xhr
0,/crm/note,example.com,::ffff:117.99.96.244,[],/crm/note,{},/note,http,{},False,True,[crm],False
1,/crm/note,example.com,::ffff:117.99.96.244,[],/crm/note,{},/note,http,{},False,True,[crm],False


In [131]:
data = pd.concat([data,df["isSafe"],df2],axis=1)

In [132]:
data.head(2)

,fresh,title,desc,host,user-agent,content-type,org_id,user_session_id,accept,content-length,...,ips,originalUrl,params,path,protocol,query,secure,stale,subdomains,xhr
0,False,Tina Johnson,Top recognize eat. Fact whom spend area thing ...,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70,...,[],/crm/note,{},/note,http,{},False,True,[crm],False
1,False,Clayton Cooper,As possible American many prepare four strong....,example.com,insomnia/2020.4.2,application/json,5f572820f65af8ac955b2e83,5fb27d3750b11901a35649fe,*/*,70,...,[],/crm/note,{},/note,http,{},False,True,[crm],False


In [202]:
nlp = data[['desc','isSafe']]

In [203]:
nlp['desc'].head()

0    Top recognize eat. Fact whom spend area thing ...
1    As possible American many prepare four strong....
2                  Tuesday Notes or 2 like 2 XSP Class
3    State third represent energy campaign not forg...
4    Us enjoy since. Time identify image position o...
Name: desc, dtype: object

In [204]:
nlp['isSafe'].value_counts()

True     572
False    428
Name: isSafe, dtype: int64

In [272]:
def strings(x):
    string = re.sub(r"[^A-Za-z]"," ",x)
    string = string.strip()
    return string
    

In [273]:
nlp['newdesc']=nlp['desc'].apply(lambda x:strings(x))

g:\python_and_jupytor\envs\tensor\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [274]:
nlp['newdesc']=nlp['newdesc'].str.replace('"',"")

g:\python_and_jupytor\envs\tensor\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [275]:
nlp

,desc,isSafe,newdesc
0,Top recognize eat. Fact whom spend area thing ...,1,Top recognize eat Fact whom spend area thing ...
1,As possible American many prepare four strong....,1,As possible American many prepare four strong ...
2,Tuesday Notes or 2 like 2 XSP Class,0,Tuesday Notes or like XSP Class
3,State third represent energy campaign not forg...,1,State third represent energy campaign not forg...
4,Us enjoy since. Time identify image position o...,0,Us enjoy since Time identify image position o...
...,...,...,...
995,Recently crime before five thought bit. Card f...,1,Recently crime before five thought bit Card f...
996,"<img src=1 href=1 onerror=""javascript:alert(1)...",0,img src href onerror javascript alert ...
997,Set nature they then low resource truth. Edge ...,1,Set nature they then low resource truth Edge ...
998,Push case them such face suffer. Letter middle...,1,Push case them such face suffer Letter middle...


In [276]:
nlp['isSafe'] = np.where(nlp['isSafe']==True,1,0)

g:\python_and_jupytor\envs\tensor\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [278]:
x = nlp['newdesc']
y = nlp['isSafe']

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=32)

In [279]:
tfidf1 = TfidfVectorizer()

In [280]:
x_train2 = tfidf1.fit_transform(x_train)
x_test2=tfidf1.transform(x_test)

In [281]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(class_weight={0:5,1:2})
lg.fit(x_train2,y_train)
y_pred = lg.predict(x_test2)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 61  23]
 [ 15 101]]
              precision    recall  f1-score   support

           0       0.80      0.73      0.76        84
           1       0.81      0.87      0.84       116

    accuracy                           0.81       200
   macro avg       0.81      0.80      0.80       200
weighted avg       0.81      0.81      0.81       200



In [283]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train2,y_train)

RandomForestClassifier()

In [284]:
rf.score(x_test2,y_test)

0.855

In [285]:
y_pred = rf.predict(x_test2)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 57  27]
 [  2 114]]
              precision    recall  f1-score   support

           0       0.97      0.68      0.80        84
           1       0.81      0.98      0.89       116

    accuracy                           0.85       200
   macro avg       0.89      0.83      0.84       200
weighted avg       0.87      0.85      0.85       200

